# DataFrames – Overview & Purpose

**Conventions**
- `id_cols = [experiment_id, gear_fault_desc, speedSet, load_value, combo]`
- Sensor features: `sensor{1|2}_{rms|p2p|skew|kurtosis}`
- Suffixes:
  - `_baseline` = reference (No fault, same speed/load)
  - `_delta` = absolute difference to baseline
  - `_delta_rel_prc` = relative difference to baseline in %

---

## 1) `master`  *(raw time-series)*
- **Granularity:** one time sample  
- **Format:** **wide**  
- **Key columns:** `id_cols`, `t_rel_s`, `sensor1`, `sensor2`  
- **Purpose:** source for time plots, histograms, feature computation

---

## 2) `df_long_ts`  *(time series for plotting)*
- **Granularity:** one time sample **per sensor**  
- **Format:** **long**  
- **Columns:** `id_cols`, `t_rel_s`, `sensor ∈ {sensor1,sensor2}`, `value`  
- **Purpose:** Plotly/Seaborn facets (traces, histograms, ECDF) without column wrangling

---

## 3) `features_wide`  *(aggregated features per experiment)*
- **Granularity:** one row per `experiment_id`  
- **Format:** **wide**  
- **Columns:** `id_cols` + absolute sensor features  
  (e.g., `sensor1_rms`, `sensor1_p2p`, `sensor1_skew`, `sensor1_kurtosis`, … and analogous `sensor2_*`)  
- **Purpose:** single source of truth for absolute metrics (no deltas)

---

## 4) `baseline_df`  *(No-fault reference per condition)*
- **Granularity:** one row per `(speedSet, load_value)` in **No fault**  
- **Format:** **wide** (reference columns only)  
- **Columns:** `speedSet`, `load_value`, all `*_baseline`  
- **Purpose:** merge partner to attach the correct reference to each row

---

## 5) `features_delta`  *(features + baseline + deltas)*
- **Granularity:** one row per `experiment_id`  
- **Format:** **wide**  
- **Columns:**  
  - **IDs:** `id_cols`  
  - **Absolute:** all from `features_wide`  
  - **Baseline:** all `*_baseline` (from `baseline_df`)  
  - **Deltas:** `*_delta` (all features), `*_delta_rel_prc` (typically for `rms`/`p2p`)  
- **Purpose:** comparisons/analysis, export; keep **No fault** rows (deltas ≈ 0 as sanity check)

---

## 6) `features_long`  *(tidy view for visualization)*
- **Granularity:** one measurement per `(experiment_id, sensor, feature, kind)`  
- **Format:** **long**  
- **Columns:** `id_cols`, `sensor`, `feature ∈ {rms,p2p,skew,kurtosis}`,  
  `kind ∈ {abs, baseline, delta, delta_rel_prc}`, `value`  
- **Purpose:** flexible plots (heatmaps, bars, radar, rankings) via `query(...)` & `pivot(...)`

---

## Typical Usage
- **Time plots / histograms:** `df_long_ts`  
- **Heatmaps / bar charts of deviations:** `features_long` with `kind ∈ {'delta','delta_rel_prc'}` (usually exclude *No fault*)  
- **Tabular comparison / export:** `features_delta`  
- **Reference check:** `features_delta` (No-fault rows → deltas ~ 0)

---

## Persistence (recommendation)
```python
features_delta.to_parquet("results/eda/features_delta.parquet", index=False)
features_long.to_parquet("results/eda/features_long.parquet", index=False)

### Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.express as px
from scipy.stats import skew, kurtosis

### Load data to DataFrame called master

In [ ]:
master = pd.read_parquet("../results/processed/master_clean.parquet")
master.head()

### Check if there is each combination of `load_value` and `speedSet` for each fault type

In [ ]:
fault_types = master['gear_fault_desc'].unique()
#display(fault_types)

In [ ]:
experiments = master[['experiment_id', 'speedSet', 'load_value', 'gear_fault_desc']].drop_duplicates()
#display(experiments)

In [ ]:
pd.crosstab(experiments['gear_fault_desc'], [experiments['speedSet'], experiments['load_value']])

Now I know that there is one experiment for each combination of `load_value` and `speedSet` per type of failure

### Check if order of magnitude is generally similar for data from both sensors

In [ ]:
basic_sensor_stats = master[['sensor1', 'sensor2']].agg(["min", "max", "mean", "std"])
display(basic_sensor_stats)

### Check the correlation between `sensor1`and `sensor2` for each experiment separately

In [ ]:
corr_sensors=(
    master.groupby('experiment_id')[['sensor1', 'sensor2']]
    .apply(lambda g: g['sensor1'].corr(g['sensor2']))
    .rename("corr")
    .reset_index()
)

corr_sensors.head()
len(corr_sensors[corr_sensors['corr']>0.5])

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(corr_sensors["corr"], bins=36, kde=True)
plt.title("histogram of correlation between sensor1 and sensor2")
plt.xlabel("correlation")
plt.ylabel("number of experiments")
plt.show()

Correlation between sensor 1 and sensor 2 is low (<0.4) for all experiments. 

### Plot sensor data for each cathegory (type of failure)

In [ ]:
sensors = ["sensor1","sensor2"]
id_cols = ["gear_fault_desc","combo","t_rel_s"]
df_long_ts = master[id_cols + sensors].melt(
    id_vars=id_cols,
    value_vars=sensors,
    var_name="sensor",
    value_name="value"
)

for fault in sorted(df_long_ts["gear_fault_desc"].unique()):
    sub = df_long_ts.query("gear_fault_desc == @fault")
    fig = px.line(
        sub,
        x="t_rel_s", y="value",
        color="combo",        # -> 6 klar verschiedene Farben
        facet_col="sensor" ,   # links sensor1, rechts sensor2
        title = fault
    )
    fig.update_layout(width=1100, height=300)  # optional größer
    fig.show()

#### Calculate typical coeffcients used in time series analysis:

1.**RMS & RMS rel. to No Fault in %:** (DE: Effektivwert)

2.**P2P & P2P rel. to No Fault in %:** Peak to Peak 

3.**Skewness (absolute values and difference to No Fault):** Shows how symmetric the distribution of values is around the mean value:
(0: symmetrical, >0: more extreme positive values, <0: more extreme negative values  )

4.**Kurtosis (absolute values and difference to No Fault):** Allows to compare the distribution of the signal to normal distribution:
(=3: similar to normal distribution, <3: peaked distribution, frequent or extreme outliers, >3: flatter distribution, fewer or less extreme outliers)

#### Create a DataFrame called features, containing  the parameters rms, p2p, skew, kurtosis  for all experiments

In [ ]:
def rms(x):
     return np.sqrt(np.mean(x**2))

def p2p(x):
    return np.max(x) - np.min(x)


keys = ['gear_fault_desc', 'experiment_id', 'combo', 'speedSet', 'load_value']
sensor_cols = ['sensor1', 'sensor2']

features = master.groupby(keys)[sensor_cols].agg([rms,p2p,skew,kurtosis])

features = features.reset_index()

def flatten_column(col):
    if isinstance(col, tuple):
        if col[1] == '':
            return col[0]
        else:
            return "_".join(col).strip()
    else:
        return col

features.columns = [flatten_column(col) for col in features.columns]


#features.columns = ['_'.join(col) if isinstance (col, tuple)  else col for col in features.columns ]

In [ ]:
#sns.boxplot(data=features, x="gear_fault_desc", y=("sensor1","rms"))
#plt.xticks(rotation=90)
#plt.xlabel("fault type", labelpad=20)

plt.figure(figsize=(10,5))
sns.boxplot(data=features, x="gear_fault_desc", y=("sensor1_rms"), color="lightgray")
sns.stripplot(data=features, x="gear_fault_desc", y=("sensor1_rms"), hue="experiment_id", dodge=True, size=5, alpha=0.7)
plt.xticks(rotation=90)
plt.legend(ncols=3, loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=features,
    x=('sensor1_rms'), y=('sensor1_p2p'),
    hue="gear_fault_desc",    # Farbe = Fehlerklasse
    style="load_value",       # Markerform = Last
    size="speedSet",          # Markergröße = Drehzahl
    sizes=(40, 200)           # kleinste und größte Markergröße
)
plt.title("RMS vs P2P (Sensor1) mit Load und Speed")
plt.xlabel("RMS (Sensor1)")
plt.ylabel("Peak-to-Peak (Sensor1)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Legende rechts außen
plt.tight_layout()
plt.show()

#### Create a DataFrame called baseline, containing the parameters rms, p2p, skewness, kurtosis only for the experiments with no fault

#### Create a DataFrame calles features_delta containing rms, p2p, skewness, kurtosis and the delta values for all experiments

In [ ]:
rel_cols = ["sensor1_rms","sensor1_p2p","sensor2_rms","sensor2_p2p"]
abs_cols = ["sensor1_rms","sensor1_p2p", "sensor1_skew","sensor1_kurtosis","sensor2_rms","sensor2_p2p", "sensor2_skew","sensor2_kurtosis"]

baseline = (
    features.loc[features['gear_fault_desc'] == 'No fault', ['speedSet', 'load_value','combo', *abs_cols]]
    .rename(columns = {c: f"{c}_baseline" for c in abs_cols})
)

features_delta = features.merge(baseline, on=['speedSet', 'load_value'], how='left')
features_delta.drop('combo_y', axis = 1, inplace = True)
features_delta.rename(columns = {'combo_x' : 'combo'}, inplace = True)

for c in abs_cols:
    features_delta[f"{c}_delta"]= features_delta[c] - features_delta[f"{c}_baseline"]

for c in rel_cols:
    features_delta[f"{c}_delta_rel_prc"] = ((features_delta[c] - features_delta[f"{c}_baseline"])/features_delta[f"{c}_baseline"])*100


##### Sanity check: all deltas for 'No_fault' must be 0

In [ ]:
display(features_delta[features_delta['gear_fault_desc'] == 'No fault'])


In [ ]:
features_delta.columns

#### Create a dataframe called features_long which contains all data in long format. Data from all columns except for id_cols is transfered to two columns called 'metric' and 'value'. The column names are split in sensor (sensor1, sensor2), feature (rms, p2k, skew, kurtosis) and kind  (baseline, delta, delta_rel_prc, abs)

In [ ]:
id_cols = ["experiment_id","gear_fault_desc","speedSet","load_value","combo"]

keep = (
    id_cols
    + abs_cols
    + [f"{c}_baseline" for c in abs_cols]
    + [f"{c}_delta"     for c in abs_cols]
    + [f"{c}_delta_rel_prc" for c in rel_cols]
) #list of column names that should be in features_long

features_long = features_delta[keep].melt(
    id_vars=id_cols, var_name="metric", value_name="value"
) #transforms to long dataframe with 2 columns called matric and value

parts = features_long["metric"].str.extract(
    r"^(sensor\d+)_(rms|p2p|skew|kurtosis)(?:_(baseline|delta|delta_rel_prc))?$"
) #splits the column name into sensor, feature and kind  

parts.columns = ["sensor","feature","kind"] 

features_long = pd.concat([features_long[id_cols], parts, features_long["value"]], axis=1) 

#features_long['kind'].fillna('abs', inplace = True)
features_long.fillna({'kind': 'abs'}, inplace=True)


In [ ]:
features_long.head(20)

In [ ]:
features.sort_values(by=['gear_fault_desc', 'speedSet', 'load_value'], axis = 0, inplace =True)
features.reset_index(inplace = True, drop = True)
features.head()

In [ ]:

df = features.set_index("experiment_id")

# Spalten auswählen
rms_rel_cols = df.filter(like="rms_delta_rel_prc")
p2p_rel_cols = df.filter(like="p2p_delta_rel_prc")
skew_cols = df.filter(like="skew")
kurt_cols = df.filter(like="kurt")


# dynamische Höhe (0.35 inch pro Zeile als grober Richtwert)
h = max(6, 0.35 * len(df))

# --- RMS-Heatmap (feine Skala, um 0 zentriert)
rms_vmax = np.nanpercentile(np.abs(rms_rel_cols.values), 95)  # robust gegen Ausreißer
plt.figure(figsize=(10, h))
sns.heatmap(rms_rel_cols, cmap="coolwarm", center=0, vmin=-rms_vmax, vmax=rms_vmax,
            annot=True, fmt=".2f")
plt.title("Δ% vs. No fault – RMS")
plt.xlabel("Feature")
plt.ylabel("experiment_id")
plt.tight_layout()
plt.show()

# --- P2P-Heatmap (eigene größere Skala, ggf. deckeln)
p2p_vmax = np.nanpercentile(np.abs(p2p_rel_cols.values), 95)  # z.B. ~200–400, je nach Daten
plt.figure(figsize=(10, h))
sns.heatmap(p2p_rel_cols, cmap="coolwarm", center=0, vmin=-p2p_vmax, vmax=p2p_vmax,
            annot=True, fmt=".1f")
plt.title("Δ% vs. No fault – P2P")
plt.xlabel("Feature")
plt.ylabel("experiment_id")
plt.tight_layout()
plt.show()

skew_vmax = np.nanpercentile(np.abs(skew_cols.values), 95)  # z.B. ~200–400, je nach Daten
plt.figure(figsize=(10, h))
sns.heatmap(skew_cols, cmap="coolwarm", center=0, vmin=-skew_vmax, vmax=skew_vmax,
            annot=True, fmt=".1f")
plt.title("Skewness")
plt.xlabel("Feature")
plt.ylabel("experiment_id")
plt.tight_layout()
plt.show()

kurt_vmax = np.nanpercentile(np.abs(kurt_cols.values), 95)  # z.B. ~200–400, je nach Daten
plt.figure(figsize=(10, h))
sns.heatmap(kurt_cols, cmap="coolwarm", center=0, vmin=-kurt_vmax, vmax=kurt_vmax,
            annot=True, fmt=".1f")
plt.title("Kurtosis")
plt.xlabel("Feature")
plt.ylabel("experiment_id")
plt.tight_layout()
plt.show()

In [ ]:
sub = df_long.query(
    "gear_fault_desc in ['No fault','missing tooth'] and combo == '8.33203125 / 0'"
)

# Gemeinsame Bin-Kanten und Range für alle Figuren
xmin, xmax = float(sub["value"].min()), float(sub["value"].max())
bins = 200
bw = (xmax - xmin) / bins

fig = px.histogram(
    sub,
    x="value",
    color="gear_fault_desc",               # No fault vs. Missing tooth
    facet_col="sensor",
    nbins=bins,
    barmode="overlay",
    opacity=0.55,
    histnorm = None,
    category_orders={"gear_fault_desc": ["No fault","missing tooth"]}
)
fig.update_traces(xbins=dict(start=xmin, end=xmax, size=bw))
fig.update_xaxes(range=[xmin, xmax], matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("sensor=", "")))
fig.update_layout(title="No fault vs. Missing tooth — combo 8.33203125 / 0")
fig.show()